# Metrices to measure the grouping results from weighted PCA scores

The extreme divergent trends in the reference and degraded groups are not the only result we want to capture, it can cause issues of PCs driven by single or a few features.

Therefore, how to balance the overall grouping structure and the extreme trends is important. 
It needs to think of weight assignment carefully.

## Weights and composite scores of sites

Prepare the data

In [162]:
# magic commands to make sure external modules are reloaded every complete run
%load_ext autoreload
%autoreload 2

import pandas as pd
from zci.data_process.dataframe_ops import get_block
from sklearn.preprocessing import StandardScaler

# read the merged dataframe
master = pd.read_excel("../data/processed/complete_env_taxa_chemical.xlsx", 
                      sheet_name="all_data_merged", 
                      header=[0, 1, 2], 
                      index_col=0)

master.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


block     sample_info                           environmental            \
subblock          raw                                     raw             
var          Latitude Longitude Waterbody  Year       LOI (%) MPS (Phi)   
StationID                                                                 
A10          42.90455 -82.46760       SCR  2004      3.436193  1.734259   
A23          42.56658 -82.57595       LSC  2004      3.226637  1.969984   
A27          42.56007 -82.42132       LSC  2004      2.802642  1.319519   
A28          42.54577 -82.42073       LSC  2004      3.201399  1.398687   
A29          42.51440 -82.43462       LSC  2004      6.180718  1.065748   

block                                                                      \
subblock                                                                    
var       Measured Depth (m) Temperature (oC) Velocity  at bottom (m/sec)   
StationID                                                                   
A10                      1.8            19.17                         NaN   
A23                      3.0            19.10                         NaN   
A27                      1.0            18.40                         NaN   
A28                      0.5            18.90                         NaN   
A29                      0.5            19.30                         NaN   

block                             ...           taxa                          \
subblock                          ...            raw                           
var       Water DO Bottom (mg/L)  ... Hydropsychidae      Hydrozoa  Nematoda   
StationID                         ...                                          
A10                        10.21  ...   3.203427e-16  3.203427e-16  1.727234   
A23                        10.09  ...   3.203427e-16  3.203427e-16  2.427993   
A27                        10.30  ...   3.203427e-16  3.203427e-16  1.802219   
A28                        12.80  ...   3.203427e-16  3.203427e-16  2.771800   
A29                         9.70  ...   3.203427e-16  3.203427e-16  4.616441   

block                                                                \
subblock                                                              
var       Oligochaeta Other Trichoptera   Sphaeriidae   Turbellaria   
StationID                                                             
A10          6.576942      3.203427e-16  3.203427e-16  3.203427e-16   
A23          5.872905      3.203427e-16  1.022139e+00  5.997595e-01   
A27          6.402280      1.580169e+00  1.317615e+00  9.964067e-01   
A28          5.867874      2.049287e+00  2.863547e+00  3.203427e-16   
A29          5.181664      3.203427e-16  3.203427e-16  3.203427e-16   

block     2008_results                          
subblock   DR_clusters       corridor_clusters  
var         DR_cluster if_RF  corridor_cluster  
StationID                                       
A10                  0     0                 1  
A23                  0     0                 1  
A27                  0     0                 1  
A28                  0     0                 1  
A29                  0     0                 1  

[5 rows x 59 columns]

Standardize the stressor features: z-score

In [163]:
# take the stressor data block
stressor = get_block(master, block="chemical", subblock= "raw")

# transform the stressor features: standardization (z-score)
scaler = StandardScaler()
stressor_standardized = pd.DataFrame(scaler.fit_transform(stressor), 
                       columns=stressor.columns, 
                       index=stressor.index)

Set weights for groups of variables

In [164]:
# import the chemical weighting utilities
from zci.sediment_pollution_assessment.chemical_weights import (
    build_weights_for_columns, # function to build weight mapping for variables
    VARIABLE_TYPE_BY_NAME, # default category mapping for: variable name -> variable type
)

# define weights for different variable types
chem_cols = stressor.columns.tolist()

# specify custom weights for certain types (others will use defaults)
std_weight = 1

type_weights = {
    "Trace Metal (pollutant)":  1 * std_weight,
    "Hydrocarbon pollutant": 1 * std_weight,        # Highest priority: petroleum/chlorobenzenes
    "organochlorine pesticide": 1 * std_weight,     # Highest priority: POPs
    "Sum of all PCBs": 1 * std_weight,              # Highest priority: PCBs
    "Binding agent": 1 * std_weight,               # Medium priority: affects bioavailability
    "Earth element (nontoxic)": 1 * std_weight   # Lowest priority: background elements
}

# Build the final weight map for the variables for later use in weighted PCA
custom_weight_map = build_weights_for_columns(chem_cols,
                                              variable_type_by_name =VARIABLE_TYPE_BY_NAME,
                                              type_weights=type_weights,
                                              weights_by_name={"p,p'-DDD": 5 * std_weight,
                                                               "As": 1 * std_weight} # set a subtle weight for Mg specifically
                                             )

The pipeline to produce: X, weights -> composite scores

In [165]:
from zci.sediment_pollution_assessment.weighted_pca import WeightedPCA_Scores

weighted_PCA_grader = WeightedPCA_Scores(
    custom_weight_map,
    weight_threshold=3.0,
    group_thresholds=(0.2, 0.8)
)

# compute the weighted PCA scores
scores_with_labels =weighted_PCA_grader.fit_transform(stressor_standardized)


=== Selected Principal Components ===
PC       Explained Var   High-Weighted Variable Loadings
--------------------------------------------------------------------------------
PC1      0.5311          p,p'-DDD: 0.926
--------------------------------------------------------------------------------
Total selected PCs: 1
High-weighted variables (1): p,p'-DDD


## PERMANOVA from skbio library

In [166]:
import numpy as np
from skbio.stats.distance import permanova, DistanceMatrix
# compute the distance matrix
from scipy.spatial.distance import pdist, squareform

X = get_block(master, block="chemical", subblock= "raw").dropna(axis = 1)

# Filter out columns with high weights (>= weight_threshold)
# high_weight_cols = [col for col, weight in custom_weight_map.items() if weight >= weighted_PCA_grader.weight_threshold]
# X = X.drop(columns=high_weight_cols)
# print(high_weight_cols)
X = np.log1p(X + 1)  # log-transform to reduce skewness
groups = scores_with_labels['pollution_quality']
dist_mat = squareform(pdist(X, metric='euclidean'))

# Convert to skbio DistanceMatrix
dm = DistanceMatrix(dist_mat, ids=X.index)

# perform PERMANOVA
permanova_results = permanova(dm, groups, permutations=999)
print(permanova_results)

method name               PERMANOVA
test statistic name        pseudo-F
sample size                     104
number of groups                  3
test statistic            18.352077
p-value                       0.001
number of permutations          999
Name: PERMANOVA results, dtype: object


## Measure the representativeness of the selected PCs

In [167]:
from zci.sediment_pollution_assessment.ordination_metrices import representativeness_of_selected_PCs
from sklearn.decomposition import PCA

representativeness_of_selected_PCs(weighted_PCA_grader)

Kept variance ratio: 0.6450334507365199


In [168]:
from zci.sediment_pollution_assessment.ordination_metrices import distinguishability_of_weighted_stressors

ordinal_scores_with_labels = distinguishability_of_weighted_stressors(weighted_PCA_grader)
ordinal_scores_with_labels

var             %OC  1234-TCB  1245-TCB        Al        As        Bi  \
StationID                                                               
GL1       -0.821452 -0.928122 -0.495938 -0.672727 -1.482910  0.083280   
S1        -1.296654 -0.928122 -0.495938 -0.877860  0.093801  0.834284   
S10       -0.809194 -0.101198 -0.219945 -0.604720 -0.447091  0.500615   
S11       -0.862690  0.091811 -0.109417 -0.612524  0.128406  0.869145   
S22       -1.011827 -0.422354 -0.411612 -0.817435 -1.482986  0.823328   
S23       -0.027013  0.201873  0.080010 -0.695916 -1.482986  0.829304   
S25       -0.829218 -0.232304 -0.384526 -0.788895 -1.482986  0.643047   
S28       -0.262688 -0.928122 -0.240891 -0.330465 -1.482986  1.026517   
S36       -0.495101 -0.387277 -0.351786 -0.524896 -1.482986  0.697828   
S37       -0.718285 -0.928122 -0.071292 -0.634375 -1.482986  0.621134   
S38       -1.204262 -0.928122 -0.339795 -0.901049 -1.482986 -0.058156   
S39       -0.980359 -0.928122 -0.272603 -0.653997 -

In [169]:
from zci.sediment_pollution_assessment.ordination_metrices import groupby_aggregation

X = get_block(master, block="chemical", subblock= "raw").dropna(axis = 1)

groupby_aggregation(X, ordinal_scores_with_labels['pollution_quality'])

TypeError: 'NoneType' object is not subscriptable

In [ ]:
groupby_aggregation(X, scores_with_labels['pollution_quality'])

quality_label               reference        medium      degraded
var                                                              
%OC                mean      1.062384      2.172643      3.443072
                   std       0.457724      1.120854      1.703585
1234-TCB           mean      0.117546      0.261964      0.308547
                   std       0.242167      0.273941      0.210863
1245-TCB           mean      0.239987      1.174300      1.228325
                   std       0.227716      2.340952      1.864162
Al                 mean   2653.666667   5064.161290   9326.523810
                   std     977.386430   3736.458814   6050.347202
As                 mean      1.218481      1.918473      2.880143
                   std       1.194881      1.288590      1.106898
Bi                 mean     21.780476     18.711776      9.264936
                   std       6.707886      8.787698     12.192662
Ca                 mean  24410.190476  32147.790323  35028.095238
                   std   13088.894723  11338.229519  12484.256734
Cd                 mean      0.258832      0.351884      0.589777
                   std       0.213580      0.227386      0.688190
Co                 mean      2.902048      4.170968      5.987619
                   std       0.869887      1.727601      2.002408
Cr                 mean      5.629810      9.992661     25.108762
                   std       2.208658      4.861060     20.900234
Cu                 mean      4.865819     12.992829     39.795476
                   std       3.324336      8.176560     46.233436
Fe                 mean   5896.761905  10244.951613  15789.857143
                   std    1805.382062   5462.425221   6199.850581
HCB                mean      2.631316     16.220794      3.878289
                   std       7.843664     80.064420      6.040860
Heptachlor Epoxide mean      0.041314      0.092530      0.017199
                   std       0.095086      0.299318      0.057935
Hg                 mean      0.104163      0.301196      0.668959
                   std       0.152021      0.441419      1.330951
K                  mean    476.761905   1026.309677   1842.119048
                   std     237.503447    748.046149   1234.114005
Mg                 mean  10078.190476  14502.225806  16552.238095
                   std    4248.918270   5249.313980   7115.643463
Mn                 mean    117.099048    170.399677    254.066667
                   std      38.818538     60.299226    100.083197
Na                 mean    101.115238    128.357903    154.693333
                   std      31.143631     42.183653     48.623089
Ni                 mean      5.652810     10.321532     20.562000
                   std       1.954821      5.298848     10.076394
OCS                mean      0.425019      1.582047      1.203688
                   std       0.670895      3.921523      2.528971
Pb                 mean      2.217905      6.640290     26.061333
                   std       1.117207     11.659320     47.607995
QCB                mean      0.347805      1.240538      0.934005
                   std       0.249827      2.954421      1.129488
Sb                 mean     27.737048     17.029362      6.863031
                   std      11.403612     11.487735      9.309182
V                  mean     10.546000     16.719823     24.000476
                   std       3.606255      6.596023     10.402934
Zn                 mean     20.954571     40.282903     86.428095
                   std       8.242075     18.672950     73.755130
mirex              mean      0.089135      0.187548      0.378776
                   std       0.205931      0.298685      0.771378
p,p'-DDD           mean      0.154409      0.537583      6.018440
                   std       0.105763      0.410342     15.941488
p,p'-DDE           mean      0.243505      0.344471      1.221605
                   std       0.234221      0.316944      1.876144
total PCB          mean      1.647218   